In [1]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

# Plotting and Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Language Detection packages
# `langdetect` for detecting language
from langdetect import detect as langdetect_detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
# `langid` for an alternative language detection method
from langid import classify as langid_classify

# Text Preprocessing and NLP
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords
# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Part-of-speech tagging
from nltk import pos_tag
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer
import nltk
# Regular expressions for text pattern matching
import re

# Word Cloud generation
from wordcloud import WordCloud

In [2]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: /home/inflaton/code/CrediNews


In [3]:
from datasets import load_dataset, concatenate_datasets, Dataset

datasets = load_dataset(
    "csv",
    data_files={
        "train": [
            "dataset/train_data_1.csv",
            "dataset/train_data_2.csv",
            "dataset/train_data_3.csv",
            "dataset/train_data_4.csv",
        ],
        "test": "dataset/test_data.csv",
        "rewritten_train": [
            "dataset/rewritten_train_data_1.csv",
            "dataset/rewritten_train_data_2.csv",
            "dataset/rewritten_train_data_3.csv",
            "dataset/rewritten_train_data_4.csv",
        ],
        "rewritten_test": "dataset/rewritten_test_data.csv",
    },
)
datasets

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 54441 examples [00:01, 31171.08 examples/s]
Generating test split: 6050 examples [00:00, 29768.12 examples/s]
Generating rewritten_train split: 54441 examples [00:01, 38242.54 examples/s]
Generating rewritten_test split: 6050 examples [00:00, 40664.04 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
    rewritten_train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    rewritten_test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
})

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def create_basic_nn(vocab_size=10000, embedding_dim=128, input_length=300, dense_units=64, dropout_rate=0.5, l2_lambda=0.01):
    """Creates a simple feedforward neural network with an embedding layer and configurable dense units."""
    
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=input_length),
        Flatten(),  # Flatten the embedding output
        Dense(dense_units, activation='relu', kernel_regularizer=l2(l2_lambda)),  
        Dropout(dropout_rate),  
        Dense(dense_units // 2, activation='relu', kernel_regularizer=l2(l2_lambda)), 
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_lambda))
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model



2025-02-20 19:21:14.031180: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-20 19:21:14.144269: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740050474.190904 2448696 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740050474.206503 2448696 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 19:21:14.325969: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def train_model(train_data, val_data, epochs=10, batch_size=64, dense_units=64, dropout_rate=0.3, grid_search=False):
    """Trains a simple NN model with word embeddings."""
    seed = 42
    tf.random.set_seed(seed)
    np.random.seed(seed)

    vocab_size = 10000
    max_sequence_length = 300

    print(f"\n🚀 Training Simple NN Model with dense_units = {dense_units}, dropout_rate = {dropout_rate} 🚀\n")

    # Extract texts and labels
    train_texts = train_data["processed_full_content"]
    val_texts = val_data["processed_full_content"]
    y_train = train_data["label"]
    y_val = val_data["label"]

    # Tokenize texts
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(train_texts)

    # Convert texts to sequences and pad them
    X_train = pad_sequences(tokenizer.texts_to_sequences(train_texts), maxlen=max_sequence_length)
    X_val = pad_sequences(tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length)

    # Create and compile the model
    model = create_basic_nn(vocab_size=vocab_size, input_length=max_sequence_length)

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model
    model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1
    )

     # Evaluate model performance on validation data
    y_pred = (model.predict(X_val) > 0.5).astype(int).reshape(-1)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    #  Print evaluation metrics
    result = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1}
    print("\n🏆 Training Results:")

    for key, value in result.items():
        print(f"🔹 {key.capitalize()}: {value:.4f}")

    return result if grid_search else model

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, train_data, val_data):
    """Evaluates the trained NN model."""
    print("📊 Evaluating Model...")

    max_words = 10000
    max_sequence_length = 300

    train_texts = train_data["processed_full_content"]
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(train_texts)

    y_val = val_data["label"]
    val_texts = val_data["processed_full_content"]

    # Convert validation text into padded sequences using the same tokenizer
    X_val = pad_sequences(tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length)

    # Predict and threshold
    y_pred = (model.predict(X_val) > 0.5).astype(int).reshape(-1)

    # Compute evaluation metrics
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1: {f1:.4f}")

    # print("\n🏆 Final Evaluation Results:")
    # for key, value in {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1}.items():
    #     print(f"🔹 {key.capitalize()}: {value:.4f}")

    # return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1}


In [7]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

def do_grid_search(data):
    """Performs Grid Search for the best NN hyperparameters (dropout_rate, dense_units)."""
    
    # ✅ Now testing different dropout rates and dense units
    param_grid = {
        "dropout_rate": [0.3, 0.5],  
        "dense_units": [64, 128]  
    }
    
    results = []
    best_score = 0
    best_params = None

    for dense_units in param_grid["dense_units"]:
        for dropout_rate in param_grid["dropout_rate"]:
            print(f"\n🔍 Testing dense_units={dense_units}, dropout_rate={dropout_rate}")

            kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            fold_scores = []

            for fold, (train_idx, val_idx) in enumerate(
                kfold.split(data["processed_full_content"], data["label"].to_numpy().ravel()), 1
            ):
                print(f"\n📂 Fold {fold}")

                train_data = data.iloc[train_idx]
                val_data = data.iloc[val_idx]

                # Pass dense_units into train_model
                result = train_model(
                    train_data=train_data,
                    val_data=val_data,
                    dense_units=dense_units,
                    dropout_rate=dropout_rate,
                    grid_search=True
                )
                
                fold_scores.append(result["f1_score"])

            avg_score = np.mean(fold_scores)
            print(f"📊 Average F1-score: {avg_score:.4f}")

            if avg_score > best_score:
                best_score = avg_score
                best_params = {"dense_units": dense_units, "dropout_rate": dropout_rate}

    return best_params


In [8]:
train_data = datasets["train"].to_pandas()
val_data = datasets["test"].to_pandas()
data = pd.concat([train_data, val_data], ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60491 entries, 0 to 60490
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   label                   60491 non-null  int64 
 1   full_content            60491 non-null  object
 2   processed_full_content  60491 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [9]:
%%time
best_params = do_grid_search(data)



🔍 Testing dense_units=64, dropout_rate=0.3

📂 Fold 1

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-02-20 19:21:28.755899: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_INVALID_VALUE: invalid argument


757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8638 - loss: 0.7606 - val_accuracy: 0.9674 - val_loss: 0.3561
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9699 - loss: 0.3565 - val_accuracy: 0.9668 - val_loss: 0.3262
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9816 - loss: 0.3195 - val_accuracy: 0.9664 - val_loss: 0.3465
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9852 - loss: 0.2994 - val_accuracy: 0.9681 - val_loss: 0.3057
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9870 - loss: 0.2985 - val_accuracy: 0.9686 - val_loss: 0.3216
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9887 - loss: 0.2889 - val_accuracy: 0.9667 - val_loss: 0.3362
Epoch 7/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9890 - loss: 0.2902 - val_accuracy: 0.9631 - val_loss: 0.3568
379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9681
🔹 Precision: 0.9620
🔹 Recal

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8616 - loss: 0.7682 - val_accuracy: 0.9620 - val_loss: 0.3749
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9702 - loss: 0.3677 - val_accuracy: 0.9583 - val_loss: 0.3900
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9801 - loss: 0.3235 - val_accuracy: 0.9578 - val_loss: 0.3346
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9836 - loss: 0.2975 - val_accuracy: 0.9589 - val_loss: 0.3844
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9873 - loss: 0.2936 - val_accuracy: 0.9587 - val_loss: 0.4365
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9872 - loss: 0.3007 - val_accuracy: 0.9571 - val_loss: 0.4054
379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9578
🔹 Precision: 0.9731
🔹 Recall: 0.9293
🔹 F1_score: 0.9507

📂 Fold 3

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


...

757/757 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8516 - loss: 0.7686 - val_accuracy: 0.9661 - val_loss: 0.3488
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9683 - loss: 0.3529 - val_accuracy: 0.9655 - val_loss: 0.3761
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9802 - loss: 0.3164 - val_accuracy: 0.9681 - val_loss: 0.3839
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9840 - loss: 0.3071 - val_accuracy: 0.9649 - val_loss: 0.3266
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9861 - loss: 0.3003 - val_accuracy: 0.9663 - val_loss: 0.3721
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9885 - loss: 0.2871 - val_accuracy: 0.9674 - val_loss: 0.3264
Epoch 7/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9888 - loss: 0.2788 - val_accuracy: 0.9668 - val_loss: 0.3135
Epoch 8/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9903 - loss: 0.2809 - val_accuracy: 0.963

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8622 - loss: 0.7531 - val_accuracy: 0.9626 - val_loss: 0.3774
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9680 - loss: 0.3614 - val_accuracy: 0.9633 - val_loss: 0.3800
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9783 - loss: 0.3382 - val_accuracy: 0.9645 - val_loss: 0.3408
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9823 - loss: 0.3300 - val_accuracy: 0.9565 - val_loss: 0.3934
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9859 - loss: 0.3213 - val_accuracy: 0.9601 - val_loss: 0.3537
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9877 - loss: 0.2944 - val_accuracy: 0.9640 - val_loss: 0.3659
379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9645
🔹 Precision: 0.9582
🔹 Recall: 0.9609
🔹 F1_score: 0.9595

📂 Fold 5

🚀 Training Simple NN Model with dense_units = 128, dropout_rate = 0.5 🚀

Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8604 - loss: 0.7679 - val_accuracy: 0.9631 - val_loss: 0.3437
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9704 - loss: 0.3564 - val_accuracy: 0.9602 - val_loss: 0.3845
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9785 - loss: 0.3321 - val_accuracy: 0.9641 - val_loss: 0.3644
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9841 - loss: 0.3135 - val_accuracy: 0.9659 - val_loss: 0.2951
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9877 - loss: 0.2923 - val_accuracy: 0.9578 - val_loss: 0.3996
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9879 - loss: 0.2895 - val_accuracy: 0.9637 - val_loss: 0.3108
Epoch 7/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9893 - loss: 0.2873 - val_accuracy: 0.9637 - val_loss: 0.3583
379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9659
🔹 Precision: 0.9640
🔹 Recal

In [10]:
%%time
model = train_model(train_data, val_data, dense_units=best_params["dense_units"], dropout_rate=best_params["dropout_rate"])



🚀 Training Simple NN Model with dense_units = 128, dropout_rate = 0.5 🚀

Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


851/851 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.8697 - loss: 0.7431 - val_accuracy: 0.9633 - val_loss: 0.3448
Epoch 2/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9720 - loss: 0.3471 - val_accuracy: 0.9663 - val_loss: 0.3811
Epoch 3/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9789 - loss: 0.3338 - val_accuracy: 0.9646 - val_loss: 0.3242
Epoch 4/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9849 - loss: 0.3133 - val_accuracy: 0.9645 - val_loss: 0.3459
Epoch 5/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9860 - loss: 0.3119 - val_accuracy: 0.9668 - val_loss: 0.3375
Epoch 6/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9870 - loss: 0.3070 - val_accuracy: 0.9660 - val_loss: 0.3237
Epoch 7/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9883 - loss: 0.3043 - val_accuracy: 0.9661 - val_loss: 0.3502
Epoch 8/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9878 - loss: 0.2910 - val_accuracy: 0.967

In [11]:
model.save("results/NN_model_original.keras")

In [12]:
# load model
from tensorflow.keras.models import load_model

model2 = load_model("results/NN_model_original.keras")
model2.summary()

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_20 (Embedding)        │ (None, 300, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_20 (Flatten)            │ (None, 38400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ (None, 64)             │     2,457,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,219,333 (42.80 MB)

 Trainable params: 3,739,777 (14.27 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,479,556 (28.53 MB)

In [13]:
%%time
evaluate_model(model, train_data, val_data)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9646
Precision: 0.9645
Recall: 0.9543
F1: 0.9594
CPU times: user 7.92 s, sys: 199 ms, total: 8.12 s
Wall time: 7.1 s


In [14]:
%%time
evaluate_model(model2, train_data, val_data)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9646
Precision: 0.9645
Recall: 0.9543
F1: 0.9594
CPU times: user 7.93 s, sys: 147 ms, total: 8.08 s
Wall time: 7.28 s


In [15]:
val_data_rewritten = datasets["rewritten_test"].to_pandas()
train_data_rewritten = datasets["rewritten_train"].to_pandas()
data_rewritten = pd.concat([train_data, train_data_rewritten, val_data, val_data_rewritten], ignore_index=True)
data_rewritten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120982 entries, 0 to 120981
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   label                   120982 non-null  int64 
 1   full_content            120982 non-null  object
 2   processed_full_content  120982 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


In [16]:
%%time

evaluate_model(model, train_data, val_data_rewritten)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.7977
Precision: 0.8531
Recall: 0.6494
F1: 0.7375
CPU times: user 7.89 s, sys: 58.1 ms, total: 7.94 s
Wall time: 7.19 s


In [17]:
%%time

evaluate_model(model2, train_data, val_data_rewritten)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.7977
Precision: 0.8531
Recall: 0.6494
F1: 0.7375
CPU times: user 7.86 s, sys: 83.7 ms, total: 7.94 s
Wall time: 8.77 s


In [18]:
%%time

best_params_rewritten = do_grid_search(data_rewritten)
best_params_rewritten


🔍 Testing dense_units=64, dropout_rate=0.3

📂 Fold 1

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀



Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.8374 - loss: 0.7398 - val_accuracy: 0.9189 - val_loss: 0.4787
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9365 - loss: 0.4847 - val_accuracy: 0.9024 - val_loss: 0.5707
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.9515 - loss: 0.4953 - val_accuracy: 0.9328 - val_loss: 0.5022
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9603 - loss: 0.4963 - val_accuracy: 0.9309 - val_loss: 0.5900
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9189
🔹 Precision: 0.9436
🔹 Recall: 0.8663
🔹 F1_score: 0.9033

📂 Fold 2

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


...

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.8374 - loss: 0.7332 - val_accuracy: 0.9245 - val_loss: 0.4514
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9374 - loss: 0.4598 - val_accuracy: 0.9218 - val_loss: 0.4859
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9504 - loss: 0.4689 - val_accuracy: 0.9277 - val_loss: 0.5276
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.9601 - loss: 0.4734 - val_accuracy: 0.9234 - val_loss: 0.5937
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

🏆 Training Results:
🔹 Accuracy: 0.9245
🔹 Precision: 0.9001
🔹 Recall: 0.9307
🔹 F1_score: 0.9152

📂 Fold 5

🚀 Training Simple NN Model with dense_units = 128, dropout_rate = 0.5 🚀

Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.8500 - loss: 0.7300 - val_accuracy: 0.9301 - val_loss: 0.4583
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9385 - loss: 0.4700 - val_accuracy: 0.9223 - val_loss: 0.5104
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9509 - loss: 0.4804 - val_accuracy: 0.9156 - val_loss: 0.5627
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9596 - loss: 0.4839 - val_accuracy: 0.9197 - val_loss: 0.5723
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9301
🔹 Precision: 0.9166
🔹 Recall: 0.9242
🔹 F1_score: 0.9204
📊 Average F1-score: 0.9079
CPU times: user 4h 2min 49s, sys: 16min 36s, total: 4h 19min 25s
Wall time: 29min 31s


{'dense_units': 128, 'dropout_rate': 0.3}

In [19]:
%%time

train_data_combined = pd.concat([train_data, train_data_rewritten], ignore_index=True)
val_data_combined = pd.concat([val_data, val_data_rewritten], ignore_index=True)
model_combined = train_model(train_data_combined, val_data_combined, 
                             dense_units=best_params_rewritten["dense_units"], 
                             dropout_rate=best_params_rewritten["dropout_rate"])
model_combined.save("results/NN_model_combined.keras")


🚀 Training Simple NN Model with dense_units = 128, dropout_rate = 0.3 🚀

Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1702/1702 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.8493 - loss: 0.7068 - val_accuracy: 0.9269 - val_loss: 0.4306
Epoch 2/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9365 - loss: 0.4637 - val_accuracy: 0.9303 - val_loss: 0.4593
Epoch 3/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9466 - loss: 0.4701 - val_accuracy: 0.9243 - val_loss: 0.5353
Epoch 4/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9568 - loss: 0.4832 - val_accuracy: 0.9259 - val_loss: 0.5218
379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9269
🔹 Precision: 0.9227
🔹 Recall: 0.9091
🔹 F1_score: 0.9159
CPU times: user 13min 24s, sys: 52.7 s, total: 14min 17s
Wall time: 1min 34s


In [20]:
evaluate_model(model_combined, train_data_combined, val_data_combined)

📊 Evaluating Model...
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9269
Precision: 0.9227
Recall: 0.9091
F1: 0.9159


In [21]:
evaluate_model(model_combined, train_data_combined, val_data)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9597
Precision: 0.9605
Recall: 0.9467
F1: 0.9536


In [22]:
evaluate_model(model_combined, train_data_combined, val_data_rewritten)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.8942
Precision: 0.8849
Recall: 0.8716
F1: 0.8782


In [23]:
model_combined2 = load_model("results/NN_model_combined.keras")
model_combined2.summary()

Model: "sequential_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_41 (Embedding)        │ (None, 300, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_41 (Flatten)            │ (None, 38400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_123 (Dense)               │ (None, 64)             │     2,457,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_82 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_124 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_83 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_125 (Dense)               │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,219,333 (42.80 MB)

 Trainable params: 3,739,777 (14.27 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,479,556 (28.53 MB)

In [24]:
evaluate_model(model_combined2, train_data_combined, val_data_combined)

📊 Evaluating Model...
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9269
Precision: 0.9227
Recall: 0.9091
F1: 0.9159


In [25]:
evaluate_model(model_combined2, train_data_combined, val_data)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9597
Precision: 0.9605
Recall: 0.9467
F1: 0.9536


In [26]:
evaluate_model(model_combined2, train_data_combined, val_data_rewritten)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.8942
Precision: 0.8849
Recall: 0.8716
F1: 0.8782


In [27]:
%%time

best_params_rewritten = do_grid_search(data_rewritten)
best_params_rewritten


🔍 Testing dense_units=64, dropout_rate=0.3

📂 Fold 1

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.8452 - loss: 0.7428 - val_accuracy: 0.9211 - val_loss: 0.4668
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.9370 - loss: 0.4734 - val_accuracy: 0.9173 - val_loss: 0.4828
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.9510 - loss: 0.4806 - val_accuracy: 0.9285 - val_loss: 0.5170
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9613 - loss: 0.4817 - val_accuracy: 0.9237 - val_loss: 0.5483
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9211
🔹 Precision: 0.9534
🔹 Recall: 0.8617
🔹 F1_score: 0.9052

📂 Fold 2

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.8408 - loss: 0.7192 - val_accuracy: 0.9286 - val_loss: 0.4447
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9352 - loss: 0.4774 - val_accuracy: 0.9238 - val_loss: 0.4977
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9492 - loss: 0.4778 - val_accuracy: 0.9129 - val_loss: 0.5662
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.9605 - loss: 0.4854 - val_accuracy: 0.9172 - val_loss: 0.5498
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9286
🔹 Precision: 0.9187
🔹 Recall: 0.9181
🔹 F1_score: 0.9184

📂 Fold 3

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


...

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.8369 - loss: 0.7211 - val_accuracy: 0.9242 - val_loss: 0.4447
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9382 - loss: 0.4644 - val_accuracy: 0.9218 - val_loss: 0.4820
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9508 - loss: 0.4679 - val_accuracy: 0.9292 - val_loss: 0.4908
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9613 - loss: 0.4694 - val_accuracy: 0.9262 - val_loss: 0.5356
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9242
🔹 Precision: 0.9322
🔹 Recall: 0.8914
🔹 F1_score: 0.9114

📂 Fold 5

🚀 Training Simple NN Model with dense_units = 128, dropout_rate = 0.5 🚀

Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.8494 - loss: 0.7203 - val_accuracy: 0.9304 - val_loss: 0.4417
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9354 - loss: 0.4782 - val_accuracy: 0.9256 - val_loss: 0.4804
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9501 - loss: 0.4796 - val_accuracy: 0.9245 - val_loss: 0.5176
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9594 - loss: 0.4841 - val_accuracy: 0.9281 - val_loss: 0.5567
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9304
🔹 Precision: 0.9214
🔹 Recall: 0.9194
🔹 F1_score: 0.9204
📊 Average F1-score: 0.9132
CPU times: user 4h 10min 17s, sys: 16min 44s, total: 4h 27min 2s
Wall time: 29min 51s


{'dense_units': 128, 'dropout_rate': 0.3}

In [28]:
%%time


model_rewritten = train_model(train_data_rewritten, val_data_rewritten, 
                             dense_units=best_params_rewritten["dense_units"], 
                             dropout_rate=best_params_rewritten["dropout_rate"])
model_rewritten.save("results/NN_model_rewritten.keras")


🚀 Training Simple NN Model with dense_units = 128, dropout_rate = 0.3 🚀

Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


851/851 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.7613 - loss: 0.8937 - val_accuracy: 0.8864 - val_loss: 0.5564
Epoch 2/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9074 - loss: 0.5597 - val_accuracy: 0.8964 - val_loss: 0.6191
Epoch 3/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.9372 - loss: 0.5721 - val_accuracy: 0.8828 - val_loss: 0.7143
Epoch 4/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9544 - loss: 0.5628 - val_accuracy: 0.8691 - val_loss: 0.8268
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🏆 Training Results:
🔹 Accuracy: 0.8864
🔹 Precision: 0.8977
🔹 Recall: 0.8357
🔹 F1_score: 0.8656
CPU times: user 6min 41s, sys: 27.6 s, total: 7min 9s
Wall time: 47.6 s


In [29]:
evaluate_model(model_rewritten, train_data_rewritten, val_data_combined)

📊 Evaluating Model...
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.8961
Precision: 0.9336
Recall: 0.8209
F1: 0.8737


In [30]:
evaluate_model(model_rewritten, train_data_rewritten, val_data_rewritten)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.8864
Precision: 0.8977
Recall: 0.8357
F1: 0.8656


In [31]:
evaluate_model(model_rewritten, train_data_rewritten, val_data)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9058
Precision: 0.9740
Recall: 0.8062
F1: 0.8822
